In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import cv2
import numpy as np
import time

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
# Function to calculate percentage of oily skin
def calculate_oily_percentage(face_img):
    # Convert image to grayscale
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    
    # Apply skin segmentation (example: simple color thresholding)
    _, skin_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Calculate percentage of oily skin
    oily_percentage = (np.sum(skin_mask == 255) / (face_img.shape[0] * face_img.shape[1])) * 100
    return oily_percentage

In [5]:
# Function to map oily percentage to level
def map_to_level(oily_percentage):
    if oily_percentage < 5:
        return "Low level"
    elif oily_percentage < 10:
        return "Normal level 1"
    elif oily_percentage < 15:
        return "Middle level 2"
    else:
        return "High level"

In [6]:
# Open camera
cap = cv2.VideoCapture(0)

start_time = time.time()
oily_percentage_sum = 0
frame_count = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region
        face_img = frame[y:y+h, x:x+w]
        
        # Calculate oily percentage
        oily_percentage = calculate_oily_percentage(face_img)
        
        # Accumulate oily percentage
        oily_percentage_sum += oily_percentage
        frame_count += 1
        
        # Get oily level
        oily_level = map_to_level(oily_percentage)
        
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Display oily level and percentage value
        text = f'Oily Level: {oily_level}, Percentage: {oily_percentage:.2f}%'
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    
    # Display the resulting frame
    cv2.imshow('Skin Care Detection', frame)
    
    # Check if 10 seconds have elapsed
    if time.time() - start_time >= 10:
        # Calculate average oily percentage
        average_oily_percentage = oily_percentage_sum / frame_count
        # Get average oily level
        average_oily_level = map_to_level(average_oily_percentage)
        
        # Display the average oily level and percentage
        print(f'Average Oily Level: {average_oily_level}, Percentage: {average_oily_percentage:.2f}%')
        
        # Break the loop
        break
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Average Oily Level: High level, Percentage: 57.51%
